# Exploratory Customer Analysis / Análise Exploratória de Clientes

## Introdução
PT: Este notebook analisa dados de clientes. Por padrão, ele carrega o arquivo real `data/customer_data.csv`. Se esse arquivo não existir, cai para dados sintéticos e marca claramente todos os gráficos e saídas como 'Dados Sintéticos'.
EN: This notebook analyzes customer data. By default, it loads the real `data/customer_data.csv`. If that file is missing, it falls back to synthetic data and clearly labels all plots/outputs as 'Synthetic Data'.

---
## 1. Setup & Data Loading / Configuração & Carregamento de Dados
PT: Execute esta seção primeiro. O notebook tenta carregar `data/customer_data.csv` com colunas esperadas: `customer_id, name, email, age, gender, purchase_amount, purchase_date, category, last_login, signup_date, country, city`. Se o CSV não for encontrado, o notebook gera dados sintéticos compatíveis com esse esquema.
EN: Run this section first. The notebook attempts to load `data/customer_data.csv` with expected columns as listed above. If the CSV is not found, it generates schema-aligned synthetic data.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

CSV_PATH = os.path.join('data', 'customer_data.csv')
EXPECTED_COLUMNS = [
    'customer_id','name','email','age','gender','purchase_amount','purchase_date',
    'category','last_login','signup_date','country','city'
]

def generate_synthetic(n=1000, seed=42):
    rng = np.random.default_rng(seed)
    base_date = datetime(2023,1,1)
    genders = ['Male','Female','Other']
    categories = ['Electronics','Clothing','Home','Beauty','Grocery','Sports']
    countries = ['Brazil','USA','UK','Germany','Canada','Spain']
    cities = ['Sao Paulo','Rio de Janeiro','New York','London','Berlin','Toronto','Madrid']
    df_syn = pd.DataFrame({
        'customer_id': [f'CUST_{i:06d}' for i in range(n)],
        'name': [f'Customer {i}' for i in range(n)],
        'email': [f'customer{i}@example.com' for i in range(n)],
        'age': rng.integers(18, 80, n),
        'gender': rng.choice(genders, n, p=[0.47,0.5,0.03]),
        'purchase_amount': np.round(rng.gamma(2.0, 60.0, n), 2),
        'purchase_date': [ (base_date + timedelta(days=int(x))).date().isoformat() for x in rng.integers(0, 365, n) ],
        'category': rng.choice(categories, n),
        'last_login': [ (datetime.now() - timedelta(days=int(x))).isoformat(timespec='seconds') for x in rng.integers(0,90,n) ],
        'signup_date': [ (base_date - timedelta(days=int(x))).date().isoformat() for x in rng.integers(0, 365*3, n) ],
        'country': rng.choice(countries, n),
        'city': rng.choice(cities, n)
    })
    return df_syn

source_label = None
if os.path.exists(CSV_PATH):
    df = pd.read_csv(CSV_PATH)
    source_label = 'real'
    missing_cols = [c for c in EXPECTED_COLUMNS if c not in df.columns]
    if missing_cols:
        print(f'AVISO/NOTICE: CSV encontrado, mas faltam colunas: {missing_cols}. Continuando somente com colunas disponíveis.')
else:
    print('CSV não encontrado. Gerando dados sintéticos compatíveis com o esquema esperado... / CSV not found. Generating schema-aligned synthetic data...')
    df = generate_synthetic()
    source_label = 'synthetic'

# Type coercions and basic cleaning aligned to expected schema
for col in ['age','purchase_amount']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
for col in ['purchase_date','signup_date','last_login']:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

print(f'Fonte de dados / Data source: {source_label.upper()}')
display(df.head())


---
## 2. Schema Check & Alignment / Verificação de Esquema
PT: Esta célula valida e alinha o esquema com as colunas esperadas.
EN: This cell validates and aligns the schema with the expected columns.

In [ ]:
present = list(df.columns)
print('Colunas presentes / Present columns:', present)
missing = [c for c in EXPECTED_COLUMNS if c not in present]
extra = [c for c in present if c not in EXPECTED_COLUMNS]
print('Faltando / Missing:', missing)
print('Extras:', extra)

# Optional: create convenience columns
if 'purchase_date' in df.columns:
    df['purchase_year'] = df['purchase_date'].dt.year
    df['purchase_month'] = df['purchase_date'].dt.to_period('M').astype(str)

display(df.sample(min(5, len(df))))


---
## 3. Exploratory Analysis / Análise Exploratória
PT: As visualizações abaixo usam a fonte real quando disponível. Se os dados forem sintéticos, o título e legendas indicam 'Dados Sintéticos'.
EN: The plots below use the real source when available. If synthetic, titles and captions indicate 'Synthetic Data'.

In [ ]:
sns.set(style='whitegrid')
label_prefix = '' if source_label=='real' else '[Dados Sintéticos / Synthetic Data] ' 

fig, axes = plt.subplots(1, 2, figsize=(12,4))
if 'age' in df.columns:
    sns.histplot(df['age'].dropna(), bins=20, kde=True, ax=axes[0])
    axes[0].set_title(label_prefix + 'Distribuição de Idade / Age Distribution')
else:
    axes[0].set_visible(False)

if 'purchase_amount' in df.columns:
    sns.histplot(df['purchase_amount'].dropna(), bins=20, kde=True, ax=axes[1])
    axes[1].set_title(label_prefix + 'Distribuição de Valor de Compra / Purchase Amount Distribution')
else:
    axes[1].set_visible(False)
plt.tight_layout(); plt.show()


In [ ]:
plt.figure(figsize=(10,4))
if {'category','purchase_amount'} <= set(df.columns):
    order = df.groupby('category')['purchase_amount'].median().sort_values(ascending=False).index
    sns.boxplot(data=df, x='category', y='purchase_amount', order=order)
    plt.title(label_prefix + 'Valor de Compra por Categoria / Purchase Amount by Category')
    plt.xticks(rotation=30, ha='right')
else:
    plt.text(0.5,0.5,'category/purchase_amount ausentes / missing', ha='center')
plt.tight_layout(); plt.show()


In [ ]:
if {'purchase_date','purchase_amount'} <= set(df.columns):
    ts = (df.dropna(subset=['purchase_date'])
            .set_index('purchase_date')
            .resample('M')['purchase_amount'].sum())
    ax = ts.plot(figsize=(10,4))
    ax.set_title(label_prefix + 'Receita Mensal / Monthly Revenue')
    plt.show()
else:
    print('Sem purchase_date/purchase_amount para série temporal.')


---
## 4. Mensagens e Instruções ao Usuário / User Guidance
PT: Para usar dados reais, garanta que `data/customer_data.csv` esteja presente. O notebook informa a fonte carregada na primeira célula (REAL ou SINTÉTICO). Em modo sintético, todos os títulos exibem a marcação correspondente.
EN: To use real data, ensure `data/customer_data.csv` is present. The notebook prints the loaded source (REAL or SYNTHETIC) in the first cell. In synthetic mode, all plot titles include the appropriate label.

---
## 5. Conclusão / Conclusion
PT: As análises e visualizações estão alinhadas ao esquema do CSV real. Em falta, dados sintéticos compatíveis são usados e claramente identificados.
EN: Analyses and visuals align to the real CSV schema; when missing, compatible synthetic data is used and clearly identified.